In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

In [33]:

torch.cuda.is_available()

False

In [34]:
device = torch.device("cpu")

In [35]:
temperatures = [
    72, 73, 75, 78, 80, 82, 85, 87, 88, 85,
    81, 78, 76, 74, 73, 72, 71, 72, 74, 76,
    79, 82, 85, 88, 90, 92, 93, 92, 89, 86
]
data = []
labels = []
for t in range(26):
    data.append([[temperatures[t+i]]for i in range(4)])
    labels.append(temperatures[t+4])
data = torch.tensor(data, dtype=torch.float32)
targets = torch.tensor(labels, dtype=torch.float32)

In [64]:
class LSTM(nn.Module):
  def __init__(self, l, hl,ol):
    super().__init__()
    self.sig = nn.Sigmoid()
    self.U = nn.Linear(l, hl)
    self.W = nn.Linear(hl, hl)
    self.Ug = nn.Linear(l,hl)
    self.Wg = nn.Linear(hl,hl)
    self.Uf = nn.Linear(l,hl)
    self.Wf = nn.Linear(hl,hl)
    self.Uo = nn.Linear(l,hl)
    self.Wo = nn.Linear(hl,hl)
    self.tanh = nn.Tanh()
    self.fc = nn.Linear(hl, ol)
  def forward(self, x, st_1, ht_1):
    xt = self.sig(self.U(x) + self.W(ht_1))
    g = self.sig(self.Ug(x) + self.Wg(ht_1))
    f = self.sig(self.Uf(x) + self.Wf(ht_1))
    q = self.sig(self.Uo(x) + self.Wo(ht_1))
    s = f * st_1 + g * xt
    h = self.tanh(s) * q
    o = self.fc(h)
    return s, h, o

class UnfoldLSTM(nn.Module):
  def __init__(self, l, hl,ol) -> None:
     super().__init__()
     self.hl = hl
     self.lstm = LSTM( l, hl,ol)
  def forward(self, x):
    x = torch.permute(x, (1,0,2))
    st_1 , ht_1 = torch.zeros( self.hl).to(device), torch.zeros( self.hl).to(device)
    for i in x:
      st_1, ht_1, o = self.lstm(i, st_1, ht_1)
    return o




In [83]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

batch_size = 26  # You can adjust this as needed
# Define hyperparameters
input_size = 1  # Input size
hidden_size = 128  # Hidden layer size
output_size = 1  # Output size
sequence_length = 4  # Length of input sequences
num_epochs = 10000
learning_rate = 0.01

dataset = TensorDataset(data, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# Instantiate the LSTM model
model = UnfoldLSTM(input_size, hidden_size, output_size)
model.to(device)
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):

    for inputs, labels in dataloader:
        inputs , labels = inputs.to(device), labels.to(device)
        predictions = model(inputs)

        # Compute loss
        loss = criterion(predictions.view(-1), labels)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
predictions


Epoch [100/10000], Loss: 2958.4814
Epoch [200/10000], Loss: 937.1837
Epoch [300/10000], Loss: 263.9018
Epoch [400/10000], Loss: 86.6648
Epoch [500/10000], Loss: 51.9279
Epoch [600/10000], Loss: 46.9428
Epoch [700/10000], Loss: 46.4206
Epoch [800/10000], Loss: 46.3799
Epoch [900/10000], Loss: 32.1518
Epoch [1000/10000], Loss: 21.3116
Epoch [1100/10000], Loss: 10.7093
Epoch [1200/10000], Loss: 5.5106
Epoch [1300/10000], Loss: 3.6298
Epoch [1400/10000], Loss: 2.5596
Epoch [1500/10000], Loss: 1.9128
Epoch [1600/10000], Loss: 1.5194
Epoch [1700/10000], Loss: 1.2931
Epoch [1800/10000], Loss: 1.1577
Epoch [1900/10000], Loss: 1.0631
Epoch [2000/10000], Loss: 0.9961
Epoch [2100/10000], Loss: 0.9413
Epoch [2200/10000], Loss: 0.8539
Epoch [2300/10000], Loss: 0.7425
Epoch [2400/10000], Loss: 0.7042
Epoch [2500/10000], Loss: 0.6905
Epoch [2600/10000], Loss: 0.6468
Epoch [2700/10000], Loss: 0.5918
Epoch [2800/10000], Loss: 0.5773
Epoch [2900/10000], Loss: 0.5515
Epoch [3000/10000], Loss: 0.4869
Epoc

tensor([[73.2115],
        [81.9981],
        [80.2234],
        [76.2262],
        [89.1399],
        [86.0084],
        [73.9787],
        [87.9834],
        [87.2624],
        [77.9816],
        [90.2274],
        [82.2673],
        [85.2137],
        [72.1611],
        [81.0352],
        [73.9765],
        [76.0066],
        [91.8482],
        [72.0492],
        [88.2386],
        [85.5061],
        [85.1229],
        [91.8761],
        [78.9166],
        [93.3143],
        [71.1283]], grad_fn=<AddmmBackward0>)

In [84]:
labels

tensor([73., 82., 80., 76., 89., 86., 74., 88., 87., 78., 90., 82., 85., 72.,
        81., 74., 76., 92., 72., 88., 85., 85., 92., 79., 93., 71.])

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have NumPy arrays for your data and targets
import numpy as np
# Assuming data and targets are NumPy arrays
# data = np.random.rand(26, 4, 1)  # Replace this with your actual data
# targets = np.random.rand(26)    # Replace this with your actual targets

# data = torch.tensor(data, dtype=torch.float32)
# targets = torch.tensor(targets, dtype=torch.float32)
dataset = TensorDataset(data, targets)
batch_size = 26  # You can adjust this as needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the last time step's output
        return out

# Define the model's hyperparameters
input_size = 1
hidden_size = 64  # You can adjust this as needed
num_layers = 2    # You can adjust this as needed
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()  # Mean squared error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate
num_epochs = 10000  # You can adjust the number of epochs

for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        inputs , labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
    if epoch % 100 ==0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [1/10000], Loss: 6745.9253
Epoch [101/10000], Loss: 5055.7319
Epoch [201/10000], Loss: 4137.6597
Epoch [301/10000], Loss: 3385.8992
Epoch [401/10000], Loss: 2753.6550
Epoch [501/10000], Loss: 2221.2544
Epoch [601/10000], Loss: 1775.2551
Epoch [701/10000], Loss: 1404.6755
Epoch [801/10000], Loss: 1099.8604
Epoch [901/10000], Loss: 852.0562
Epoch [1001/10000], Loss: 653.2350
Epoch [1101/10000], Loss: 496.0274
Epoch [1201/10000], Loss: 373.7024
Epoch [1301/10000], Loss: 280.1738
Epoch [1401/10000], Loss: 210.0104
Epoch [1501/10000], Loss: 158.4461
Epoch [1601/10000], Loss: 121.3793
Epoch [1701/10000], Loss: 95.3571
Epoch [1801/10000], Loss: 77.5438
Epoch [1901/10000], Loss: 65.6722
Epoch [2001/10000], Loss: 57.9818
Epoch [2101/10000], Loss: 53.1472
Epoch [2201/10000], Loss: 50.2022
Epoch [2301/10000], Loss: 48.4668
Epoch [2401/10000], Loss: 47.4792
Epoch [2501/10000], Loss: 46.9375
Epoch [2601/10000], Loss: 46.6515
Epoch [2701/10000], Loss: 46.5065
Epoch [2801/10000], Loss: 46.4360
